In [64]:
from pystats_utils.utils.patient import Patient

import pandas as pd

data = pd.read_csv("./test/database/treatment.csv",
                   sep = "\t")
data = data[data["Cohort_Treatment"] == "yes"]
data = data[data["Cohort_Cazavi"] == "no"]

patients = []
for index, row in data.iterrows():
    patients.append(Patient(row))

In [65]:
for patient in patients:
    treatment = patient.translateTreatmentLine(only = "all",
                                               condensed = False)

    test = [i for j in treatment for i in j]

    if "Caz-avi" in test: print(patient.pt, treatment)

46 [{'Inactive'}, {'Caz-avi', 'Inactive'}, {'Caz-avi', 'Meropenem'}, {'Meropenem'}, {'Meropenem'}, {'Meropenem'}, {'Meropenem'}, {'Meropenem'}, {'Meropenem'}, {'Meropenem'}]
116 [{'Ceftriaxone'}, {'Ceftriaxone', 'Inactive'}, {'Inactive'}, {'Caz-avi', 'Inactive'}, {'Ceftriaxone', 'Caz-avi'}, {'Ceftriaxone'}, {'Ceftriaxone'}, {'Ceftriaxone'}, {'Ceftriaxone'}, {'Ceftriaxone'}, {'Ceftriaxone'}, {'Ceftriaxone', 'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}]


In [1]:
import pandas as pd
from pystats_utils.utils.patient import Patient
from pystats_utils.utils.treatment_plot import generatePlot

data = pd.read_csv("./test/database/treatment.csv",
                   sep = "\t")
data = data[data["Cohort_Treatment"] == "yes"]
#data = data[data["Mortality_31d"] == "no"]
data = data[data["Cohort_Cazavi"] == "yes"]

patients = []
for index, row in data.iterrows():

    patient = Patient(row)

    patients.append(patient)

a = generatePlot(patients, days = 30, title = "Caz-avi Cohort")

a.save('treatment_cazavi.png', dpi=300)

data = pd.read_csv("./test/database/treatment.csv",
                   sep = "\t")
data = data[data["Cohort_Treatment"] == "yes"]
#data = data[data["Mortality_31d"] == "no"]
data = data[data["Cohort_Cazavi"] == "no"]

patients = []
for index, row in data.iterrows():

    patient = Patient(row)

    patients.append(patient)

a = generatePlot(patients, days = 30, title = "BAT Cohort")
a.save('treatment_bat.png', dpi=300)


data = pd.read_csv("./test/database/treatment.csv",
                   sep = "\t")
data = data[data["Cohort_Treatment"] == "yes"]
data = data[data["Mortality_31d"] == "no"]
data = data[data["Cohort_Cazavi"] == "yes"]

patients = []
for index, row in data.iterrows():

    patient = Patient(row)

    patients.append(patient)

a = generatePlot(patients, days = 30, title = "Caz-avi Cohort alive")
a.save('treatment_cazavi_alive.png', dpi=300)


data = pd.read_csv("./test/database/treatment.csv",
                   sep = "\t")
data = data[data["Cohort_Treatment"] == "yes"]
data = data[data["Mortality_31d"] == "no"]
data = data[data["Cohort_Cazavi"] == "no"]

patients = []
for index, row in data.iterrows():

    patient = Patient(row)

    patients.append(patient)

a = generatePlot(patients, days = 30, title = "BAT Cohort alive")
a.save('treatment_bat_alive.png', dpi=300)


/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 20 x 20 in image.
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: treatment_cazavi.png
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 20 x 20 in image.
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: treatment_bat.png
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 20 x 20 in image.
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: treatment_cazavi_alive.png
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 20 x 20 in image.
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:722: PlotnineWarning: File

In [2]:
from pystats_utils.utils.patient import Patient

import pandas as pd

data = pd.read_csv("./test/database/treatment.csv",
                   sep = "\t")
data = data[data["Cohort_Treatment"] == "yes"]
data = data[data["Cohort_Cazavi"] == "no"]

patients = []
keys = []
for index, row in data.iterrows():

    patient = Patient(row)

    patients.append(patient)

    treatment = patient.translateTreatmentLine(mergeInactive = True,
                                               condensed = False,
                                               start = patient.bloodculture,
                                               days = 30)

    if "Colistin" in (atbs := [element for day in treatment for element in day]) and "Amikacin" in atbs:
        keys.append(patient)

In [24]:
keys

In [3]:
from datetime import datetime, timedelta

for key in keys:
    treatment = key.translateTreatmentLine(mergeInactive = False,
                                        condensed = False,
                                        start = key.bloodculture,
                                        days = 30)

    print("Patient", key.pt, key.__dict__)
    for index, day in enumerate(treatment):
        print(index, key.firstDay + timedelta(days=index), day)
    print("_______________________________________")

Patient 3 {'pt': 3, 'center': 'fmusp', 'center_code': 'brasil_2', 'cohort_global': 'yes', 'cohort_treatment': 'yes', 'cohort_cazavi': 'no', 'cohort_bat': 'yes', 'cohort_combi': 'no', 'cazavi_avaibility': 'neutral', 'clinical_success_14d': 'yes', 'clinical_success_30d': 'yes', 'time2death_31d': 31, 'mortality_31d': 'no', 'bloodculture': datetime.datetime(2017, 8, 22, 0, 0), 'carbapenemase': 'kpc', 'guidelines': 'clsi', 'treatment': [Amikacin: S
	2017-08-22 00:00:00 -> 2017-09-01 00:00:00, Colistin: S
	2017-08-22 00:00:00 -> 2017-09-17 00:00:00, Meropenem: R
	2017-08-24 00:00:00 -> 2017-09-17 00:00:00, Tigecycline: R
	2017-09-10 00:00:00 -> 2017-09-17 00:00:00]}
0 2017-08-22 00:00:00 {'Amikacin_S', 'Colistin_S'}
1 2017-08-23 00:00:00 {'Amikacin_S', 'Colistin_S'}
2 2017-08-24 00:00:00 {'Meropenem_R', 'Amikacin_S', 'Colistin_S'}
3 2017-08-25 00:00:00 {'Meropenem_R', 'Amikacin_S', 'Colistin_S'}
4 2017-08-26 00:00:00 {'Meropenem_R', 'Amikacin_S', 'Colistin_S'}
5 2017-08-27 00:00:00 {'Meropen